**1. Upload**

Seleção do dataset exportado pelo Roboflow

In [ ]:
from google.colab import files
print("Faça upload do dataset .zip exportado do Roboflow")
uploaded = files.upload()

**2. Extração**

Extração dos diretórios e arquivos do dataset.zip importado.



In [ ]:
import zipfile

dataset_zip = list(uploaded.keys())[0]
dataset_folder = "Cutwatch_dataset"

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(dataset_folder)

!ls Cutwatch_dataset

**3. Validação e Definição**

Validação do conteúdo extraído e definição dos caminhos para os nomes das classes e imagens de treinamento e validaçao.


In [ ]:
!cat Cutwatch_dataset/data.yaml

**4. Treinamento**

Instalação e utilização do pacote Utralytics para executar o treinamento do modelo 'yolov8n.pt' e gerar o arquivo best.pt

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

!yolo detect train model=yolov8n.pt data=/content/Cutwatch_dataset/data.yaml epochs=1 imgsz=640

**5. Visualização**

Visualização dos pesos treinados e disponíveis em runs/detect/train/weights

In [ ]:
!ls /content/runs/detect/train/weights

**6. Download**

Download do modelo treinado `best.pt` que será usado como base para o fine-tuning

In [ ]:
from google.colab import files
files.download("/content/runs/detect/train/weights/best.pt")

**7. Vídeo para Teste**

Upload dos vídeos fornecidos para extração dos frames.

In [ ]:
from google.colab import files
uploaded = files.upload()

**8. Extração de frames**

Função onde um frame é extraído e salvo como .jpg conforme a configuração dos parâmetros (padrão 1 segundo).

    :param video_path: Caminho do vídeo.
    :param output_folder: Pasta onde os frames serão salvos.
    :param frame_interval_s: Intervalo de tempo em segundos entre frames extraídos.

In [ ]:
!pip install opencv-python-headless
import cv2
import os

def extrair_frames(video_path, output_folder, frame_interval_s=3):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * frame_interval_s)

    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"✅ Extração concluída: {saved_count} frames salvos em '{output_folder}'")

**9. Executa a função**

A função será executada para cada vídeo e as imagens dos frames serão extraídas para o diretório com o mesmo nome do vídeo.


In [ ]:
os.makedirs("frames", exist_ok=True)

for video_filename in uploaded.keys():
    folder_name = os.path.join("frames", f"frames_{os.path.splitext(video_filename)[0]}")
    print(f"📂 Processando: {video_filename}")
    extrair_frames(video_filename, folder_name, frame_interval_s=1)

**10. Download**

Um arquivo .zip será gerado para que seja possível realizar o download das imagens geradas.

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("video_frames", 'zip', "frames")

# Download the zip file
files.download("video_frames.zip")

**11. Acesso ao Drive**

Permisão para acessar os arquivos armazenados no seu Google Drive, como o best.pt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**12. Upload dos Arquivos**

Selecione o dataset do novo projeto gerado e o arquivo `best.pt` (/content/runs/detect/train/weights/) gerado no treinamento do modelo.

In [ ]:
from google.colab import files
print("Envie os arquivos: Cutwatch_dataset_2.zip e best.pt")
uploaded = files.upload()

**13. Extração**

Extração dos diretórios e arquivos do dataset.zip importado.

In [ ]:
import zipfile

dataset_zip = "Cutwatch_dataset_2.zip"
dataset_folder = "Cutwatch_dataset_2"

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(dataset_folder)

!ls Cutwatch_dataset_2

**14. Validação**

Validação do conteúdo extraído e definição dos caminhos para os nomes das classes e imagens de treinamento e validaçao.

In [ ]:
from pathlib import Path

with open("Cutwatch_dataset_2/data.yaml", "w") as f:
    f.write("""train: /content/Cutwatch_dataset_2/train/images
val: /content/Cutwatch_dataset_2/valid/images

names:
  0: objeto_cortante
""")

**15. Labels inválidos**

Substitui qualquer classe diferente de 0 por 0 em todos os arquivos .txt de treino e validação

In [ ]:
from pathlib import Path
def corrigir_labels_para_classe_zero(label_dir):
    for txt_file in Path(label_dir).glob("*.txt"):
        linhas_corrigidas = []
        with open(txt_file, "r") as f:
            for linha in f:
                partes = linha.strip().split()
                if len(partes) == 5:
                    partes[0] = "0"  # força classe 0
                    linhas_corrigidas.append(" ".join(partes))
        with open(txt_file, "w") as f:
            f.write("\n".join(linhas_corrigidas) + "\n")

corrigir_labels_para_classe_zero("/content/Cutwatch_dataset_2/train/labels")
corrigir_labels_para_classe_zero("/content/Cutwatch_dataset_2/valid/labels")

print("✅ Labels corrigidos: todas as anotações agora usam classe 0.")

**16. Treinamento**

Instalação e utilização do pacote Utralytics na versão até 8.3.40 compatível com exportação e executar o treinamento do modelo 'yolov8n.pt' utilizando o modelo best.pt como base e para um novo arquivo best.pt

In [ ]:
!pip install roboflow
!pip install "ultralytics<=8.3.40"

!yolo detect train model=best.pt data=Cutwatch_dataset_2/data.yaml epochs=5 imgsz=640

**17. Modelo TFLite**

Exportação do modelo treinado para o arquivo .tflite que será utilizado na aplicação

In [ ]:
!yolo export model=runs/detect/train/weights/best.pt format=tflite